In [1]:
# coding:utf-8
from torch import nn
from torchvision import models
# coding:utf-8
import numpy as np
import torch
import random
import logging
import numpy as np
import argparse
import os

import torchvision
import wandb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR
from torchvision import models
from torchvision import transforms
from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from torch.utils.data.sampler import SequentialSampler
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

D:\anaconda3\envs\py37\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Create the input data pipeline
parser = argparse.ArgumentParser()

parser.add_argument("--total_epoch", default=15, type=int, help="Total number of training epochs to perform.")
parser.add_argument('--train_dir', default='D:\\label_mapping\\src_data\\datas\\train_pre',
                    help="Directory containing the dataset")
parser.add_argument('--test_dir', default='D:\\label_mapping\\src_data\\datas\\test_pre',
                    help="Directory containing the dataset")
parser.add_argument('--model_dir', default='D:\\label_mapping\\new_models', help="Directory containing params.json")
parser.add_argument('--restore_file', default="best_67_", help="Optional, name of the file in --model_dir containing weights to reload before \
                    training")  # 'best' or 'train'
parser.add_argument('--charset_size', default=67, help="Choose the first `charset_size` character to conduct our "
                                                       "experiment.")
parser.add_argument('--image_size', default=112, help="Needs to provide same value as in training.")
parser.add_argument('--batch_size', default=256, help='')

args = parser.parse_args(args=[])
args.device = device
net = models.resnet50(pretrained=True)
net.fc = nn.Linear(net.fc.in_features, 67)
net = net.to(args.device)


D:\anaconda3\envs\py37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
D:\anaconda3\envs\py37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
def load_data(args):

    transform = transforms.Compose([
        transforms.Resize((args.image_size, args.image_size)),  # 调整图像大小
        transforms.ToTensor(),  # 将图像转换为张量
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化图像张量
    ])
    logging.info("开始加载数据...")
    test_dataset = torchvision.datasets.ImageFolder(root=args.test_dir, transform=transform)
    samplerC = SequentialSampler(test_dataset)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False,sampler=samplerC)
    return test_loader


def eval(args, model, test_loader,map_num,restore_file):
    restore_path = os.path.join(
        args.model_dir, restore_file +'.pth.tar')
    logging.info("Restoring parameters from {}".format(restore_path))
    if not os.path.exists(restore_path):
        raise("File doesn't exist {}".format(restore_path))
    checkpoint = torch.load(restore_path,map_location={'cuda:3':'cuda:0','cuda:2':'cuda:0','cuda:1':'cuda:0'})
    model.load_state_dict(checkpoint['state_dict'])


    eval_loss = 0.0
    total_acc = 0.0
    model.eval()
    loss_function = torch.nn.CrossEntropyLoss()
    num = 0
    loss_list = []
    labels_list = []
    labels_src_list=[]
    preds_list = []
    total = 0
    for images, l in tqdm(test_loader):
        #num=num+1
        labels_src_list.extend(l)
        images = images.to(device)
        labels = map(l, map_num)
        # end=time.time()
        # t=end-start
        labels = torch.round(labels) # linux环境下需要注释该行代码
        labels = labels.long() # linux环境下需要注释该行代码
        labels = labels.to(device)
        total += labels.size(0)
        with torch.no_grad():
            logits = model(images)  # model返回的是（bs,num_classes）和weight
            batch_loss = loss_function(logits, labels)
            # 记录误差
            eval_loss += batch_loss.item()
            # 记录准确率
            _, preds = logits.max(1)
            num_correct = (preds == labels).sum().item()
            # print("pred:", preds)
            # print("num_correct:", num_correct)
            total_acc += num_correct
        preds = preds.cpu().numpy()
        loss = batch_loss.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        loss_list.append(loss)
        labels_list.extend(labels)
        preds_list.extend(preds)
        #if num==15: break

    loss = eval_loss / len(test_loader)
    acc = total_acc / (len(test_loader) * args.batch_size)
    print(preds_list)
    print("正确个数为："+str(total_acc)+",总个数为："+str(total))
    return loss, acc, preds_list,labels_list,labels_src_list


def map(label, i):
    l = label % 67
    c = label / 67
    f = torch.floor(c)  # linux环境下需要注释该行代码
    label = (l + f * i) % 67
    return label

In [4]:
test_loader = load_data(args)
loss0, acc0, preds_list0,labels_list0,labels_src_list0=eval(args, net, test_loader, 0,"best_67_0_9421")
loss1, acc1, preds_list1,labels_list1,labels_src_list1=eval(args, net, test_loader, 1,"best_67_1_9434")
loss2, acc2, preds_list2,labels_list2,labels_src_list2=eval(args, net, test_loader, 2,"best_67_2_9420")
loss3, acc3, preds_list3,labels_list3,labels_src_list3=eval(args, net, test_loader, 3,"best_67_3_9426")
loss4, acc4, preds_list4,labels_list4,labels_src_list4=eval(args, net, test_loader, 4,"best_67_4_9408")
loss5, acc5, preds_list5,labels_list5,labels_src_list5=eval(args, net, test_loader, 5,"best_67_5_9406")
loss6, acc6, preds_list6,labels_list6,labels_src_list6=eval(args, net, test_loader, 6,"best_67_6_9433")
loss7, acc7, preds_list7,labels_list7,labels_src_list7=eval(args, net, test_loader, 7,"best_67_7_9435")
print(acc0,acc1,acc2,acc3,acc4,acc5,acc6,acc7)

开始加载数据...
Restoring parameters from D:\label_mapping\new_models\best_67_0_9421.pth.tar
100%|██████████| 440/440 [10:54<00:00,  1.49s/it]

[0, 0, 0, 0, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 30, 1, 1, 1, 1, 1, 1, 30, 1, 1, 19, 1, 1, 1, 30, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 42, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 49, 55, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 50, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 42, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 5, 10, 10, 10, 10, 10, 10

In [9]:
labels_lm=[]
for i in range(len(preds_list0)):
    label_lm = []
    label_lm.append(preds_list0[i])
    label_lm.append(preds_list1[i])
    label_lm.append(preds_list2[i])
    label_lm.append(preds_list3[i])
    label_lm.append(preds_list4[i])
    label_lm.append(preds_list5[i])
    label_lm.append(preds_list6[i])
    label_lm.append(preds_list7[i])
    labels_lm.append(label_lm)

112604
[2, 58, 47, 36, 25, 14, 3, 59]


In [11]:
def map1(label, i):
    l = label % 67
    c = label / 67
    f = np.floor(c)  # linux环境下需要注释该行代码
    label = (l + f * i) % 67
    return int(label)
ground_truth=[]
for l_src in range(3755):
    gth=[]
    gth.append(map1(l_src,0))
    gth.append(map1(l_src,1))
    gth.append(map1(l_src,2))
    gth.append(map1(l_src,3))
    gth.append(map1(l_src,4))
    gth.append(map1(l_src,5))
    gth.append(map1(l_src,6))
    gth.append(map1(l_src,7))
    ground_truth.append(gth)

[0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[2, 2, 2, 2, 2, 2, 2, 2]
[3, 3, 3, 3, 3, 3, 3, 3]
[4, 4, 4, 4, 4, 4, 4, 4]
[5, 5, 5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7, 7, 7]
[8, 8, 8, 8, 8, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 9, 9]
[10, 10, 10, 10, 10, 10, 10, 10]
[11, 11, 11, 11, 11, 11, 11, 11]
[12, 12, 12, 12, 12, 12, 12, 12]
[13, 13, 13, 13, 13, 13, 13, 13]
[14, 14, 14, 14, 14, 14, 14, 14]
[15, 15, 15, 15, 15, 15, 15, 15]
[16, 16, 16, 16, 16, 16, 16, 16]
[17, 17, 17, 17, 17, 17, 17, 17]
[18, 18, 18, 18, 18, 18, 18, 18]
[19, 19, 19, 19, 19, 19, 19, 19]
[20, 20, 20, 20, 20, 20, 20, 20]
[21, 21, 21, 21, 21, 21, 21, 21]
[22, 22, 22, 22, 22, 22, 22, 22]
[23, 23, 23, 23, 23, 23, 23, 23]
[24, 24, 24, 24, 24, 24, 24, 24]
[25, 25, 25, 25, 25, 25, 25, 25]
[26, 26, 26, 26, 26, 26, 26, 26]
[27, 27, 27, 27, 27, 27, 27, 27]
[28, 28, 28, 28, 28, 28, 28, 28]
[29, 29, 29, 29, 29, 29, 29, 29]
[30, 30, 30, 30, 30, 30, 30, 30]
[31, 31, 31, 31, 31, 31, 31, 31]
[32, 32, 32, 32, 32, 32,

In [12]:
def minus(a,b,inter=67):
    while b>a:b=b-inter
    return a-b
def add(a,b,inter=67):
    while a+b>=inter: b=b-inter
    return a+b
def pattern(l1,l2):
    num=0
    for i in range(len(l1)):
        if l1[i]==l2[i]:num+=1
    return num

In [1]:
def judge8(label):
    length=len(label)
    flag=[0,0,0,0,0,0]
    for i in range(length-1):
        a=minus(label[i+1],label[i])
        #if a>56:break
        grth1=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth1)>=4:return grth1
        elif pattern(label,grth1)>=3:
            flag[0]=1
            break
    for i in range(length-2):
        if minus(label[i+2],label[i])%2!=0:break
        a=int(0.5*minus(label[i+2],label[i]))
        #if a>56:break
        grth2=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth2)>=4:return grth2
        elif pattern(label,grth2)>=3:
            flag[1]=1
            break
    for i in range(length-3):
        if minus(label[i+3],label[i])%3!=0:break
        a=int(minus(label[i+3],label[i])/3)
        #if a>56:break
        grth3=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth3)>=4:return grth3
        elif pattern(label,grth3)>=3:
            flag[2]=1
            break
    for i in range(length-4):
        if minus(label[i+4],label[i])%4!=0:break
        a=int(minus(label[i+4],label[i])/4)
        #if a>56:break
        grth4=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth4)>=4:return grth4
        elif pattern(label,grth4)>=3:
            flag[3]=1
            break
    for i in range(length-5):
        if minus(label[i+5],label[i])%5!=0:break
        a=int(minus(label[i+5],label[i])/5)
        #if a>56:break
        grth5=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth5)>=4:return grth5
        elif pattern(label,grth5)>=3:
            flag[4]=1
            break
    for i in range(length-6):
        if minus(label[i+6],label[i])%6!=0:break
        a=int(minus(label[i+6],label[i])/6)
        #if a>56:break
        grth7=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth7)>=4:return grth7
        elif pattern(label,grth7)>=3:
            flag[5]=1
            break
    for i in range(length-7):
        if minus(label[i+7],label[i])%7!=0:break
        a=int(minus(label[i+7],label[i])/7)
        #if a>57:break
        grth8=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
        if pattern(label,grth8)>=4:return grth8
        elif pattern(label,grth8)>=3:
            flag[5]=1
            break
    if 1 in flag:
        if flag[0]==1:return grth1
        if flag[1]==1:return grth2
        if flag[2]==1:return grth3
        if flag[3]==1:return grth4
        if flag[4]==1:return grth5
        if flag[5]==1:return grth7
        if flag[6]==1:return grth8
    i=random.randint(0,5)
    a=minus(label[i+1],label[i])
    # while a>56:
    #     i=random.randint(0,4)
    #     if minus(label[i+2],label[i])%2!=0:break
    #     a=int(0.5*minus(label[i+2],label[i]))
    grth6=[minus(label[i],i*a),add(label[i],(-i+1)*a),add(label[i],(-i+2)*a),add(label[i],(-i+3)*a),add(label[i],(-i+4)*a),add(label[i],(-i+5)*a),add(label[i],(-i+6)*a),add(label[i],(-i+7)*a)]
    return grth6

In [42]:
results=[]
for i in range(len(labels_lm)):
    res=judge8(labels_lm[i])
    a=minus(res[1],res[0])
    src_res=67*a+res[0]
    results.append(src_res)
# print(len(results))
# print(len(labels_src_list0))
cor=0
dirlist=[]
photolist=[]
for i in range(int(len(results))):
    if results[i]==labels_src_list0[i]: cor=cor+1
    # else:print(int(i/30),i%30)
    # dirlist.append(int(i/30))
    # photolist.append(i%30)
    #print(results[i*67],labels_src_list0[i*67])
print("test集正确个数：{},总数：{}".format(cor,int(len(results))))
print("acc:{}".format(cor/int(len(results))))
# for i in range(int(3755)):
#     print(ground_truth[i],results[i*29],labels_src_list0[i*29])

0.942080965909091 0.9433682528409091 0.9420010653409091 0.9426047585227273 0.9408114346590909 0.9405628551136364
[9, -201, -143, -85, -27, 31, 22, 13]
[8, 27, 46, 65, 17, 36, 55, 7]
[48, 28, 8, 55, 35, 15, 62, 42]
[22, -6, 33, 5, 44, 16, 55, 27]
[57, -94, -44, 6, 56, 39, 22, 5]
[32, -49, 4, 57, 43, 29, 15, 1]
[31, -15, 6, 27, 48, 2, 23, 44]
[62, 47, 32, 17, 2, 54, 39, 24]
[15, 25, 35, 45, 55, 65, 8, 18]
[52, 58, 64, 3, 9, 15, 21, 27]
[38, 6, 41, 9, 44, 12, 47, 15]
[21, 25, 29, 33, 37, 41, 45, 49]
[31, 53, 8, 30, 52, 7, 29, 51]
[23, -75, -39, -3, 33, 2, 38, 7]
[44, -164, -104, -44, 16, 9, 2, 62]
[9, -81, -37, 7, 51, 28, 5, 49]
[24, 57, 23, 56, 22, 55, 21, 54]
[2, 8, 14, 20, 26, 32, 38, 44]
[0, -29, 9, 47, 18, 56, 27, 65]
[33, 44, 55, 66, 10, 21, 32, 43]
[12, -128, -67, -6, 55, 49, 43, 37]
[57, -12, 53, 51, 49, 47, 45, 43]
[4, -84, -38, 8, 54, 33, 12, 58]
[28, 22, 16, 10, 4, 65, 59, 53]
[12, -33, -11, 11, 33, 55, 10, 32]
[57, 2, 14, 26, 38, 50, 62, 7]
[9, -133, -74, -15, 44, 36, 28, 20]
